# CNN Model to classify a disease based on the chest x-ray

In [1]:
import numpy as np
import pandas as pd

In [6]:
data_entry_file_name = "Data_Entry_2017.csv"
bbox_list_file_name = "BBox_List_2017.csv"
data_entry_df  = pd.read_csv(data_entry_file_name)
bbox_df = pd.read_csv(bbox_list_file_name)

In [3]:
data_entry_df

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168


In [ ]:
data_entry_df

In [12]:
data_entry_df.rename(columns={"OriginalImage[Width": "ImageWidth", "Height]": "ImageHeight", 
                              "OriginalImagePixelSpacing[x": "ImgPixelSpacingX", "y]": "ImgPixelSpacingY"})

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,ImageWidth,ImageHeight,ImgPixelSpacingX,ImgPixelSpacingY
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168


In [15]:
data_entry_set = set(data_entry_df['Finding Labels'])
data_entry_set

{'Atelectasis',
 'Atelectasis|Cardiomegaly',
 'Atelectasis|Cardiomegaly|Consolidation',
 'Atelectasis|Cardiomegaly|Consolidation|Edema',
 'Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Infiltration',
 'Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Infiltration|Mass',
 'Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Infiltration|Mass|Nodule',
 'Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Mass',
 'Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Pleural_Thickening',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion|Infiltration',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion|Infiltration|Mass',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion|Infiltration|Mass|Pleural_Thickening',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion|Infiltration|Nodule|Pleural_Thickening',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion|Infiltration|Pneumonia',
 'Atelectasis|Cardiomegaly|Consolidation|Effusion|M

In [8]:
bbox_label_set = set(bbox_df['Finding Label'])

In [17]:
data_entry_df['Finding Labels'].value_counts()

No Finding                                                          60361
Infiltration                                                         9547
Atelectasis                                                          4215
Effusion                                                             3955
Nodule                                                               2705
                                                                    ...  
Cardiomegaly|Edema|Effusion|Pneumonia                                   1
Atelectasis|Cardiomegaly|Effusion|Infiltration|Mass|Pneumothorax        1
Cardiomegaly|Consolidation|Edema|Effusion|Pleural_Thickening            1
Edema|Effusion|Emphysema|Infiltration                                   1
Edema|Infiltration|Mass|Nodule|Pneumonia                                1
Name: Finding Labels, Length: 836, dtype: int64

In [ ]:
bbox_label_set = set(bbox_df['Finding Label'])

In [9]:
bbox_label_set

{'Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltrate',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax'}

In [11]:
bbox_df.rename(columns={"Bbox [x": "X", "y": "Y", "w": "Width", "h]": "Height"})

,Image Index,Finding Label,X,Y,Width,Height
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525
2,00029817_009.png,Atelectasis,221.830509,317.053115,155.118644,216.949152
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034
4,00017877_001.png,Atelectasis,660.067797,569.780786,200.677966,78.101695
...,...,...,...,...,...,...
979,00029464_015.png,Atelectasis,198.940451,352.900746,615.537778,323.128889
980,00025769_001.png,Atelectasis,701.838229,572.491858,103.537778,63.715556
981,00016837_002.png,Atelectasis,140.913785,658.962969,271.928889,94.435556
982,00020124_003.png,Atelectasis,175.047118,580.456302,244.622222,103.537778
